In [ ]:
import io
import json
import os
import random
import subprocess
import time
from collections import defaultdict
from datetime import datetime
from pathlib import Path

import numpy
import pytesseract
from IPython.display import clear_output, display
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter
from tqdm import tqdm

In [ ]:
import cv2
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def shell(cmd, debug=False, returns_POpen=False, close_fds=False):
    if debug:
        print(cmd)

    if returns_POpen:
        process = subprocess.Popen(cmd, close_fds=close_fds)
        return process

    process = subprocess.run(cmd, stdout=subprocess.PIPE)
    return process.stdout.decode("utf-8").strip().splitlines()

In [ ]:
def adb(cmd, debug=False, returns_POpen=False, close_fds=False):
    # This function runs adb commands on your connected device or emulator.
    if type(cmd) == str:
        cmd = cmd.split(" ")
    cmd = ["adb"] + cmd
    return shell(cmd, debug=debug, returns_POpen=returns_POpen, close_fds=close_fds)


def TypeTextOnPhone(text):
    adb(["shell", "input", "text", text.replace(" ", "%s")])
    adb(["shell", "input", "keyevent", "66"])


adb("wait-for-device")

In [84]:
def sleep(num):
    s = random.choice([-1.75, -1,-0.5,0, 0.5, 0.75, 1])
    num += s
    print(f"sleeping for: {num}")
    time.sleep(num)

In [ ]:
def pullPhoneScreen(resize_ratio=None, as_numpy=False, print_times=False):
    s = time.time()
    adb("shell screencap -p /sdcard/screen.png")
    adb("pull /sdcard/screen.png ./game.png")
    adb("shell rm /sdcard/screen.png")
    im = Image.open("game.png")
    im = im.convert("RGB")
    if resize_ratio is not None:
        im = im.resize(
            (int(im.width * resize_ratio), int(im.height * resize_ratio)),
            Image.Resampling.LANCZOS,
        )
    if print_times:
        print("pull image took ", time.time() - s)
    if as_numpy:
        return numpy.array(im)
    return im

In [ ]:
def PillowToCv2(img):
    img = np.array(img)
    im = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return im

In [ ]:
class Beholder_Layer:
    def __init__(self, name, data, offsets=None):
        self.offsets = offsets
        self.name = name
        self.data = data

    def show(self):
        img = cv2.cvtColor(self.data, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(img))

    def chop(self, new_name, x, y, h, w):
        return Beholder_Layer(
            name=new_name, data=self.data[y : y + h, x : x + w], offsets=(x, y)
        )

In [ ]:
class Beholder_Layer_Chopper:
    def __init__(self, name):
        self.name = name

    def run(self, bh):
        pass

In [ ]:
class Beholder_Matcher:
    def __init__(self, name, layer, data):
        self.name = name
        self.layer = layer
        self.data = data

    def show(self):
        img = cv2.cvtColor(self.data, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(img))

    def __repr__(self):
        return f"BW: {self.name}"

In [ ]:
class Beholder:
    def __init__(self, videoFrameGenerator):
        self.generator = videoFrameGenerator
        self.matchers = {}
        self.threshhold = 0.95
        self.layers = {}
        self.layer_modifiers = []

    def addLayerModifer(self, ch: Beholder_Layer_Chopper):
        self.layer_modifiers.append(ch)

    def addDefaultMatcher(self, name, filename, layer="gray", convertToGray=True):
        if Path(filename).exists():

            img = PillowToCv2(Image.open(filename))
            if convertToGray:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            self.matchers[name] = Beholder_Matcher(name, layer, img)
        else:
            print("File is missing")

    def readNextImage(self):
        self.layers["image"] = Beholder_Layer(
            name="image", data=PillowToCv2(self.generator())
        )

    def digestImage(self):
        for m in self.layer_modifiers:
            try:
                self.layers.update(m.run(self))
            except Exception as e:
                print(e)

    def findMatches(self, matchers=None):
        if matchers is None:
            matchers = self.matchers
        self.readNextImage()
        self.digestImage()
        matches = defaultdict(list)
        for m in matchers:
            m = matchers[m]

            result = cv2.matchTemplate(
                self.layers[m.layer].data, m.data, method=cv2.TM_CCOEFF_NORMED
            )
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if max_val > self.threshhold:
                height, width = m.data.shape[:2]
                layer_offset_y = 0
                layer_offset_x = 0
                if self.layers[m.layer].offsets is not None:
                    layer_offset_x, layer_offset_y = self.layers[m.layer].offsets
                top_left = max_loc
                center = (
                    (layer_offset_x + top_left[0] + (width / 2)),
                    (layer_offset_y + top_left[1] + (height / 2)),
                )

                threshold = 0.8
                loc = np.where(result >= threshold)

                f = set()
                sensitivity = 60
                for pt in zip(*loc[::-1]):
                    f.add(
                        (
                            round(layer_offset_x + pt[0] / sensitivity),
                            layer_offset_y + round(pt[1] / sensitivity),
                        )
                    )

                found_count = len(f)
                matches[m.name].append((m, center, found_count))

        return dict(matches)

In [ ]:
class Beholder_Layer_Chopper_Grayscale(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer):
        self.name = name
        self.from_layer = from_layer

    def run(self, bh: Beholder):
        o = {}
        o[self.name] = Beholder_Layer(
            name=self.name,
            data=cv2.cvtColor(bh.layers[self.from_layer].data, cv2.COLOR_BGR2GRAY),
        )
        return o

In [ ]:
class Beholder_Layer_Tesseract(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer):
        self.name = name
        self.from_layer = from_layer

    def run(self, bh: Beholder):
        o = {}

In [ ]:
class Beholder_Layer_Chopper_AtCord(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer, x, y, h, w, extract_text=False):
        self.name = name
        self.from_layer = from_layer
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.extract_text = extract_text

    def run(self, bh: Beholder):
        o = {}
        o[self.name] = Beholder_Layer(
            name=self.name,
            data=bh.layers[self.from_layer].data[
                self.y : self.y + self.h, self.x : self.x + self.w
            ],
            offsets=(self.x, self.y),
        )
        if self.extract_text:
            img = Image.fromarray(bh[self.from_layer].data)
            img = img.convert("L")  # grayscale
            img = img.filter(ImageFilter.MedianFilter())  # a little blur
            img = img.point(lambda x: 0 if x < 140 else 255)  # threshold (binarize)

            txt = pytesseract.image_to_string(img)
            o[f"{self.name}_Text"] = txt.splitlines()
        return o

In [ ]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image"))

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory", from_layer="gray", x=1903, y=355, w=2222 - 1903, h=905 - 355
    )
)

# Below is Auto-Resmith


In [81]:
a.matchers = {}
a.addDefaultMatcher(
    name="Button",
    layer="gray",
    filename="./templates/ui/Begin_project.png",
    convertToGray=True,
)
a.addDefaultMatcher(
    name="UnfinishedSmithing",
    layer="inventory",
    filename="./templates/inventory/forge_item.png",
)
a.addDefaultMatcher(
    name="FinishedArrow",
    layer="gray",
    filename="./templates/ui/FinishedArrow.png",
    convertToGray=True,
)

time_since_heat = time.time()
Button_count = 0
Anvil_count = 0
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        print(matches)
        if "UnfinishedSmithing" in matches:
            Button_count = 0
            Anvil_count = 0
            if time_since_heat != 0:
                print(int(time.time() - time_since_heat), "since heating")
            if time.time() - time_since_heat > 30:
                print("time for heating")
                adb(f"shell input tap 1192 150")
                sleep(5)
                print("returning to anvil")
                adb(f"shell input tap 1550 525")
                time_since_heat = time.time()
            sleep(7)
            continue
        if "FinishedArrow" in matches:
            center = matches["FinishedArrow"][0][1]
            adb(f"shell input tap {center[0]} {center[1]}")
        if "Button" in matches:
            if Button_count == 0:
                center = matches["Button"][0][1]
                adb(f"shell input tap {center[0]} {center[1]}")
                Button_count += 1
            else:
                print("Cannot continue smithing.")
                a.layers["image"].show()
                break

    else:
        if Anvil_count == 0:
            print("opening anvil menu")
            adb(f"shell input tap 1550 525")
            time_since_heat = 0
            Anvil_count += 1
        else:
            print("Cannot continue smithing.")
            a.layers["image"].show()
            break
    sleep(3.75)

{'UnfinishedSmithing': [(BW: UnfinishedSmithing, (2185.0, 628.0), 2)]}
2 since heating
sleeping for 6
{'UnfinishedSmithing': [(BW: UnfinishedSmithing, (2185.0, 628.0), 2)]}
10 since heating
sleeping for 9
{'UnfinishedSmithing': [(BW: UnfinishedSmithing, (2185.0, 628.0), 2)]}
22 since heating
sleeping for 6
{'UnfinishedSmithing': [(BW: UnfinishedSmithing, (2185.0, 628.0), 2)]}
30 since heating
time for heating
sleeping for 6
returning to anvil
sleeping for 7


KeyboardInterrupt: 

# Auto Smelt Ores

In [76]:
a.matchers = {}

a.addDefaultMatcher(
    name="close_btb",
    layer="gray",
    filename="./templates/ui/crafting_dialog_close_button.png",
    convertToGray=True,
)
a.addDefaultMatcher(
    name="Button",
    layer="gray",
    filename="./templates/ui/Begin_project.png",
    convertToGray=True,
)
Button_count = 0
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        print(matches)
        if "close_btb" in matches:
            Button_count = 0
            print("Currently Crafting")
            sleep(5)
            continue
        else:
            if "Button" in matches:
                if Button_count == 0:
                    center = matches["Button"][0][1]
                    adb(f"shell input tap {center[0]} {center[1]}")
                    Button_count += 1
                else:
                    print("Cannot continue smelting.")
                    break
    else:
        adb(f"shell input tap 1150 780")
        sleep(3)
        adb(f"shell input tap 518 1029")
    sleep(5)

{'Button': [(BW: Button, (1772.0, 1016.0), 1)]}
sleeping for 4
{'close_btb': [(BW: close_btb, (1495.5, 225.5), 1)]}
Currently Crafting
sleeping for 6
{'close_btb': [(BW: close_btb, (1495.5, 225.5), 1)]}
Currently Crafting
sleeping for 3
sleeping for 3
sleeping for 6
{'Button': [(BW: Button, (1772.0, 1016.0), 1)]}
sleeping for 7
sleeping for 1
sleeping for 7
{'Button': [(BW: Button, (1772.0, 1016.0), 1)]}
Cannot continue smelting.


# Below is Auto Feathers on Bolts

In [67]:
a.matchers = {}
a.addDefaultMatcher(
    name="bolts",
    layer="gray",
    filename="./templates/inventory/mith_bolts.png",
    convertToGray=True,
)
a.addDefaultMatcher(
    name="fletch",
    layer="gray",
    filename="./templates/ui/fletch.png",
    convertToGray=True,
)
a.addDefaultMatcher(
    name="close_btb",
    layer="gray",
    filename="./templates/ui/crafting_dialog_close_button.png",
    convertToGray=True,
)

in_menu = False
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "close_btb" in matches:
            print("Currently Crafting")
            time.sleep(2)
            continue
        if in_menu:
            if "fletch" in matches:
                center = matches["fletch"][1]
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = False
        else:
            if "bolts" in matches:
                center = matches["bolts"][1]
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = True
    time.sleep(2)

KeyboardInterrupt: 

# Auto OreChest/NotePaper/Mining Reset

In [ ]:
a.matchers = {}
a.addDefaultMatcher(
    name="paper",
    layer="inventory",
    filename="./templates/inventory/items/notepaper.png",
)
a.addDefaultMatcher(
    name="ore",
    layer="inventory",
    filename="./templates/ore/oricalhalcite_inventory.png",
)
a.addDefaultMatcher(
    name="orebox", layer="inventory", filename="./templates/ore/box2.png"
)

s = time.time()
ore_box_full = False

start_mining = 0
previous_ore_count = 0
while True:
    if time.time() - start_mining >= 20:
        print("reset mining")
        adb(f"shell input tap 1215 830")
        start_mining = time.time()
    matches = a.findMatches()
    if len(matches.keys()) > 0:
        if "ore" in matches:
            if previous_ore_count != matches["ore"][0][2]:
                print(f"{matches['ore'][0][2]} ores in inventory")
                previous_ore_count = matches["ore"][0][2]
            if matches["ore"][0][2] >= 10:
                print(f"Took {time.time() - s} Seconds to mine 10 ore")

                s = time.time()
                if not ore_box_full:
                    if "orebox" in matches:
                        print("oreboxing")
                        center = matches["orebox"][0][1]
                        adb(f"shell input tap {center[0]} {center[1]}")
                        time.sleep(1)
                        matches = a.findMatches()
                        if "ore" in matches.keys():
                            print("I think the ore box is full!")
                            ore_box_full = True
                        else:
                            start_mining = 0
                            continue

                if "ore" in matches:
                    print("papering")
                    center = matches["ore"][0][1]
                    adb(f"shell input tap {center[0]} {center[1]}")
                    time.sleep(0.5)
                    center = matches["paper"][0][1]
                    adb(f"shell input tap {center[0]} {center[1]}")
                    start_mining = 0
                    time.sleep(2)
    if start_mining != 0:
        time.sleep(10)

# Other Stuff

In [ ]:
time.time() - s

In [ ]:
447 / 20

In [71]:
a.readNextImage()
a.digestImage()
for item in a.layers.keys():
    print(item, a.layers[item].data.shape)

image (1080, 2400, 3)
gray (1080, 2400)
inventory (550, 319)


In [ ]:
for item in a.matchers:
    print(item.name, item.data.shape)

In [ ]:
matches = a.findMatches()
print(matches.keys())

In [ ]:
a.layers["gray"].data.shape